# MuJoCo入门篇04. 代码设计的方法论Methodology
https://zhuanlan.zhihu.com/p/705594503

In [1]:
import mujoco
import mediapy as media
import matplotlib.pyplot as plt

import time
import os
import itertools
import numpy as np

rel_path = "eg01p5_5_scene.xml"
dir_name = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
xml_path = os.path.join(dir_name + '\\' + rel_path)
print(xml_path)
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)

# renderer = mujoco.Renderer(model, 480, 480)
# mujoco.mj_forward(model, data)
# renderer.update_scene(data, "fixed")
# media.show_image(renderer.render())


n_frames = 120
height = 480
width = 640
frames = []
fps = 60.0
times = []
sensordata = []

renderer = mujoco.Renderer(model, height, width)

# constant actuator signal
mujoco.mj_resetData(model, data)
data.ctrl = 40

# ctrl: real(mjModel.nu), “0 0 …”
# Vector of controls, copied into mjData.ctrl when the simulation state is set to this keyframe.

# simulate and render
for i in range(n_frames):
  while data.time < i/fps:#time的micro step应该比1/fps更精细,所以在一帧之内,需要step多次, make sense. 
    mujoco.mj_step(model, data)
    times.append(data.time)
    # sensordata.append(data.sensor('accelerometer').data.copy())
  renderer.update_scene(data, "fixed")
  frame = renderer.render()
  frames.append(frame)

data.ctrl = -40
for j in range(n_frames, 2*n_frames):
  while data.time < j/fps:
    mujoco.mj_step(model, data)
    times.append(data.time)
  renderer.update_scene(data, "fixed")
  frame = renderer.render()
  frames.append(frame)
    
media.show_video(frames, fps=fps)

C:\Users\wenbin.li\Documents\GitHub\MuJoCo-Tutorial-fork\tutorial\eg01_p5_5_sw2urdf\urdf\eg01p5_5_scene.xml


This browser does not support the video tag.


In [1]:
import mujoco as mj
from mujoco.glfw import glfw
import mujoco.viewer
import numpy as np
import time
import os

class BallControl:
    def __init__(self, filename, is_show): #对应伪代码 initVisualData();
        # 1. model and data 加载模型与数据
        self.model = mj.MjModel.from_xml_path(filename)
        self.data = mj.MjData(self.model)
        self.is_show = is_show
        if self.is_show:
            self.viewer = mujoco.viewer.launch_passive(self.model, self.data, key_callback=self.keyboard_cb, show_left_ui = True, show_right_ui = False)
            """
            check doc->python->interactive viewer -> passive viewer
            By calling viewer.launch_passive(model, data). This function does not block, allowing user code to continue execution. 
            In this mode, the user’s script is responsible for timing and advancing the physics state, 
            and mouse-drag perturbations will not work unless the user explicitly synchronizes incoming events.
            """
            
            """
            check doc-> API ref->Types->StructTypes->Visualization->mjvCamera & mjvOption
            """
            self.viewer.opt.frame = mj.mjtFrame.mjFRAME_WORLD
            #self.viewer.opt.frame = mj.mjtFrame.mjFRAME_CAMERA
            """
            MuJoCo objects whose spatial frames can be rendered.
            """
            #self.viewer.cam.type = 'mjCAMERA_FIXED'
            self.viewer.cam.fixedcamid = 0
            # self.viewer.cam.lookat = [0.0, 0.0, 0.0]
            # #self.viewer.cam.trackbodyid = 1
            # self.viewer.cam.distance = 4.0
            # self.viewer.cam.azimuth = 90
            # self.viewer.cam.elevation = -45
        # 2. init Controller
        # self.init_controller()

    def init_controller(self): #对应伪代码 initControlData();
        # 1. set init pos, vel 设置初始化条件(位置和速度)
        # self.data.qpos[0] = 0.0
        self.data.qvel[0] = -5.0
        self.data.qvel[1] = 0.0
        self.data.qvel[2] = 0.0
        self.data.qvel[3] = 0.0
        self.data.qvel[4] = 0.0
        self.data.qvel[5] = 0.0
        # 2. set the controller, 加载控制器
        mj.set_mjcb_control(self.controller)

    def controller(self, model, data): #对应伪代码 callback function (此处按照controller与visualization所规定的callback函数进行补充)
        """
        This controller adds drag force to the ball
        The drag force has the form of
        F = (cv^Tv)v / ||v||
        """
        vx, vy, vz = data.qvel[0], data.qvel[1], data.qvel[2]
        v = np.sqrt(vx * vx + vy * vy + vz * vz)
        c = 1.0
        data.qfrc_applied[0] = -c * v * vx
        data.qfrc_applied[1] = -c * v * vy
        data.qfrc_applied[2] = -c * v * vz
        # data.qfrc_applied[0] = 3.2
        # data.qfrc_applied[1] = 0
        # data.qfrc_applied[2] = 0
        data.qfrc_applied[3] = 0.0
        data.qfrc_applied[4] = 0.0
        data.qfrc_applied[5] = 0.0

    def main(self): 
        """
        对应伪代码 realtime simulation,  
        while(...) {
        updateControlData();
        updateVisualData(); }
        """
        sim_start, sim_end = time.time(), 50.0
        while time.time() - sim_start < sim_end:
            step_start = time.time()
            loop_num, loop_count = 50, 0
            # 1. running for 0.002*50 = 0.1s
            while loop_count < loop_num:
                loop_count = loop_count + 1
                mj.mj_step(self.model, self.data)
            # 2. GUI show
            if self.is_show:
                if self.viewer.is_running():
                    self.viewer.cam.lookat[0] = self.data.qpos[0]
                    self.viewer.sync()
                else:
                    break
            # 3. sleep for next period
            step_next_delta = self.model.opt.timestep * loop_count - (time.time() - step_start)
            if step_next_delta > 0:
                time.sleep(step_next_delta)
        if self.is_show: 
            """
            对应伪代码 4. end simulation, 
            deleteControlData(); 
            deleteVisualData();
            """
            self.viewer.close()

    def keyboard_cb(self, keycode):
        if chr(keycode) == ' ':
            # mj.mj_resetData(self.model, self.data)
            # ^comment this line to prevent reseting to origin everytime keyboard_callback
            mj.mj_forward(self.model, self.data)
            self.init_controller()

if __name__ == "__main__":
    rel_path = "car_1.xml"
    # dir_name = os.path.dirname(__file__)
    dir_name = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
    xml_path = os.path.join(dir_name + "\\" + rel_path)
    print(xml_path)
    is_show = True
    ballControl = BallControl(xml_path, is_show)
    ballControl.main()

C:\Users\wenbin.li\Documents\GitHub\MuJoCo-Tutorial-fork\tutorial\eg01_p5_5_sw2urdf\urdf\car_1.xml


In [1]:
import mujoco as mj
from mujoco.glfw import glfw
import mujoco.viewer
import numpy as np
import time
import os

class BallControl:
    def __init__(self, filename, is_show):
        # 1. model and data
        self.model = mj.MjModel.from_xml_path(filename)
        self.data = mj.MjData(self.model)
        self.is_show = is_show
        if self.is_show:
            self.viewer = mujoco.viewer.launch_passive(self.model, self.data, key_callback=self.keyboard_cb)
            self.viewer.opt.frame = mj.mjtFrame.mjFRAME_WORLD
            self.viewer.cam.fixedcamid = 0
            # self.viewer.cam.lookat = [0.0, 0.0, 0.0]
            # self.viewer.cam.distance = 8.0
            # self.viewer.cam.azimuth = 90
            # self.viewer.cam.elevation = -45
        # 2. init Controller
        # self.init_controller()

    def init_controller(self):
        # 1. set init pos, vel
        # self.data.qpos[0] = 0.0
        self.data.qvel[0] = 0.0
        self.data.qvel[1] = 0.0
        self.data.qvel[2] = 0.0
        self.data.qvel[3] = 0.0
        self.data.qvel[4] = 0.0
        self.data.qvel[5] = 0.0
        # 2. set the controller
        mj.set_mjcb_control(self.controller)

    def controller(self, model, data):
        """
        This controller adds drag force to the ball
        The drag force has the form of
        F = (cv^Tv)v / ||v||
        """
        vx, vy, vz = data.qvel[0], data.qvel[1], data.qvel[2]
        v = np.sqrt(vx * vx + vy * vy + vz * vz)
        c = 1.0
        data.qfrc_applied[0] = -c * v * vx
        data.qfrc_applied[1] = -c * v * vy
        data.qfrc_applied[2] = -c * v * vz
        data.qfrc_applied[0] = 0
        data.qfrc_applied[1] = 0
        data.qfrc_applied[2] = 0
        data.qfrc_applied[3] = 0.0
        data.qfrc_applied[4] = 0.0
        data.qfrc_applied[5] = 1.0

    def main(self):
        sim_start, sim_end = time.time(), 50.0
        while time.time() - sim_start < sim_end:
            step_start = time.time()
            loop_num, loop_count = 50, 0
            # 1. running for 0.002*50 = 0.1s
            while loop_count < loop_num:
                loop_count = loop_count + 1
                mj.mj_step(self.model, self.data)
            # 2. GUI show
            if self.is_show:
                if self.viewer.is_running():
                    self.viewer.cam.lookat[0] = self.data.qpos[0]
                    self.viewer.sync()
                else:
                    break
            # 3. sleep for next period
            step_next_delta = self.model.opt.timestep * loop_count - (time.time() - step_start)
            if step_next_delta > 0:
                time.sleep(step_next_delta)
        if self.is_show:
            self.viewer.close()

    def keyboard_cb(self, keycode):
        if chr(keycode) == ' ':
            # mj.mj_resetData(self.model, self.data)
            # ^comment this line to prevent reseting to origin everytime keyboard_callback
            mj.mj_forward(self.model, self.data)
            self.init_controller()

if __name__ == "__main__":
    rel_path = "eg01p5_5_scene.xml"
    # dir_name = os.path.dirname(__file__)
    dir_name = os.path.dirname(__file__) if '__file__' in globals() else os.getcwd()
    xml_path = os.path.join(dir_name + "\\" + rel_path)
    print(xml_path)
    is_show = True
    ballControl = BallControl(xml_path, is_show)
    ballControl.main()

C:\Users\wenbin.li\Documents\GitHub\MuJoCo-Tutorial-fork\tutorial\eg01_p5_5_sw2urdf\urdf\eg01p5_5_scene.xml
